#Installation:

We are installing these following libraries for our machine learning project - Handwriting Detection using MNISt dataset. 
We wanted to have an interactive window where the user can write down numbers and have the model predict its value. This is 
achieved with the help of keras and tensorflow (for the machine learning part) and pygame and open-cv (for the input window 
and writing part).

In [ ]:
!pip install pygame
!pip install keras
!pip install tensorflow 
!pip install opencv-python

Here, we import the required modules for input window, array conversion of pixel values of the input images, plotting the required 
accuracy and loss values during training, and the rest for loading the model and dataset and to train them.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras.datasets import mnist

%matplotlib inline

We have loaded the training and testing data into their respective variables. Now we will see the shape of the training model and
then make our calculations based on that.

In [ ]:
(train_x,train_y),(test_x,test_y) = mnist.load_data()
train_x.shape


We will now plot an image, located at index 1 of the training dataset and map it to grayscale.

In [ ]:
plt.imshow(train_x[1],cmap='gray')

In [ ]:
train_y

Normalize and reshaping our X data. We have converted the pixels to decimal values and then saveit for hot encoding.

In [ ]:
train_x = train_x.reshape(-1,28,28,1)
test_x  = test_x.reshape(-1,28,28,1)

train_x = train_x.astype('float32')
test_x  = test_x.astype('float32')

train_x = train_x/255
test_x  = test_x/255


one hot encode our y data. Using the method to_categorical(), a numpy array (or) a vector which has integers that represent 
different categories, can be converted into a numpy array (or) a matrix which has binary values and has columns equal to the 
number of categories in the data.


In [ ]:
from keras.utils import np_utils 
train_y = np_utils.to_categorical(train_y)
test_y  = np_utils.to_categorical(test_y)

Again checking for the made changes.

In [ ]:
train_y[1]

#Creating our model

A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor
and one output tensor.

Now fixing the input shape as required. In grayscale format and 28x28 pixels

In [ ]:
input_shape=(28,28,1)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Flatten,Dropout,Dense,MaxPooling2D
from tensorflow.keras.optimizers import SGD
model = Sequential()

Keras Conv2D is a 2D Convolution Layer, this layer creates a 
convolution kernel that is wind with layers input which helps produce a tensor of outputs.

In [ ]:
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape,padding='SAME'))
model.add(MaxPooling2D(pool_size=(2,2)))

It’s just a thing function that you use to get the output of node. It is also known as Transfer Function.

It is used to determine the output of neural network like yes or no. It maps the resulting values in between 
0 to 1 or -1 to 1 etc. (depending upon the function).

In [ ]:
model.add(Conv2D(64,(3,3),activation='relu',padding='SAME'))

The tf.layers.maxPooling2d() function is used to apply max pooling operation on spatial data.

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

Keras dropout is a mechanism that helps reduce odds while overfitting for every epoch of the model by following the method
of dropping, skipping the neurons present in the neural network in a random fashion. When the approach followed is of minibatch,
then this dropping or skipping of a neuron is carried out for every individual minibatch.

In [ ]:
model.add(Dropout(0.25))

Flatten is used to flatten the input. For example, if flatten is applied to layer having input shape as
(batch_size, 2,2), then the output shape of the layer will be (batch_size, 4)

In [ ]:
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

Compiling the data on the basis of 'accuracy' metrics.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

#model.compile(optimizer ="rmsprop", loss ="categorical_crossentropy",metrics =['accuracy'])

#Training our data
In terms of artificial neural networks, an epoch refers to one cycle through the full training dataset. 
Usually, training a neural network takes more than a few epochs.


In [ ]:
#batch_size=32
epochs=10
#from keras.callbacks import EarlyStopping
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlystop=EarlyStopping(monitor='val_acc',min_delta=0.01, patience=4, verbose=1)
modelcheck=ModelCheckpoint("./MNIST_file.h5",monitor='val_accuracy', verbose=1, save_best_only=True)
callback=[earlystop,modelcheck]
plotting_data = model.fit(train_x,
                          train_y,
                          validation_split=0.3, 
                          callbacks=callback,
                          epochs=50,
                          verbose=1,
                          validation_data=(test_x,test_y))
#plotting_data = model.fit(train_x, train_y, validation_split=0.3,callbacks=EarlyStopping(monitor='val_loss', verbose=1))
#verbose set to 1 for only watching the training metrics, loss and precision data.
loss,accuracy = model.evaluate(test_x,test_y,verbose=0)


In [ ]:
print('Test loss ---> ',str(round(loss*100,2)) +str('%'))
print('Test accuracy ---> ',str(round(accuracy*100,2)) +str('%'))

#Plotting the required loss and accuracy data with epochs being the value being plotted against.

In [ ]:
plotting_data_dict = plotting_data.history

test_loss = plotting_data_dict['val_loss']
training_loss = plotting_data_dict['loss']
test_accuracy = plotting_data_dict['val_accuracy']
training_accuracy = plotting_data_dict['accuracy']

epochs = range(1,len(test_loss)+1)

plt.plot(epochs,test_loss,marker='X',label='test_loss')
plt.plot(epochs,training_loss,marker='X',label='training_loss')
plt.legend()

In [ ]:
plt.plot(epochs,test_accuracy,marker='X',label='test_accuracy')
plt.plot(epochs,training_accuracy,marker='X',label='training_accuracy')
plt.legend()

#Save the model to the file and then load for using it to predict the written values from the user.

In [ ]:
model.save('MNIST_file.h5')

print('Model Saved !!!')


#This code block uses openc-cv to make the input window. Now we take in input from the user.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras.datasets import mnist

%matplotlib inline
classifier = load_model('bestmodel.h5')

In [ ]:
drawing=False
cv2.namedWindow('drawing board')
black_image = np.zeros((256,256,3),np.uint8)
ix,iy=-1,-1

def draw_circles(event,x,y,flags,param):
    global ix,iy,drawing
    if event== cv2.EVENT_LBUTTONDOWN:
        drawing=True
        ix,iy=x,y
        
    elif event==cv2.EVENT_MOUSEMOVE:
        if drawing==True:
            cv2.circle(black_image,(x,y),5,(255,255,255),-1)
            
    elif event==cv2.EVENT_LBUTTONUP:
        drawing = False
        
cv2.setMouseCallback('drawing board',draw_circles)

while True:
    
    cv2.imshow('drawing board',black_image)
    if cv2.waitKey(1)==27:
        break
    elif cv2.waitKey(1)==13:
        input_img = cv2.resize(black_image,(28,28))
        input_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
        input_img = input_img.reshape(1,28,28,1)
        res=(model.predict(input_img,1,verbose=0)[0])
        #res = classifier.predict_classes(input_img,1,verbose=0)[0]
        #print(res.index(1))
        #print(np.where(res==1))
        cv2.putText(black_image,text=str(np.where(res==1)[0][0]),org=(205,30),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(255,255,255),thickness=2)
    elif cv2.waitKey(1)==ord('c'):
        black_image = np.zeros((256,256,3),np.uint8)
        ix,iy=-1,-1
cv2.destroyAllWindows()


Now, we will set few keys for functionality for the drawing board. The keys are as follows:

**mouse button hold** : To draw the number on the black window.

**mouse button up**:   To let the code crop out the image and keep the image saved as buffer for prediction.

**c**:                 Clear the drawing board.

**Enter** :             To display the predicted value after the model runs over the drawing.

**Esc**:               To exit the window and terminated the program.
